In [1]:
import torchvision.transforms as transforms
from torchvision.datasets import ImageFolder
from torchvision.models import efficientnet_b0, EfficientNet_B0_Weights
from torch.utils.data import DataLoader
from torcheval.metrics import R2Score
import pandas as pd
import torch
from torch.utils.data import random_split
import shutil
import os
from dataset import PGLSDataset
from models import PGLSModel, SimpleTabularModel, EnsemblePGLSModel
import timm
from torchrec.models import deepfm

/home/szaryvip/.local/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print('Using device:', device)

Using device: cuda


In [3]:
for folder in ["data/train_images", "data/test_images"]:
    if "0" not in os.listdir(folder):
        print("Moving images to 0 folder")
        os.makedirs(f"{folder}/0")
        for filename in os.listdir(folder):
            if filename.lower().endswith(".jpeg"):
                source_path = os.path.join(folder, filename)
                target_path = os.path.join(f"{folder}/0", filename)

                shutil.move(source_path, target_path)

In [4]:
transform = transforms.Compose([
    transforms.Resize((288, 288)),  # Resize the image to 224x224
    transforms.ToTensor(),         # Convert PIL image to tensor (H x W x C) in the range [0.0, 1.0]
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])  # Normalize the image
])

batch_size = 16     # use 4 if problem with GPU memory

In [5]:
train_images_path = 'data/train_images'
test_images_path = 'data/test_images'

train_csv_path = 'data/train.csv'
test_csv_path = 'data/test.csv'


tabular_data = pd.read_csv(train_csv_path)
targets = ["X4", "X11", "X18", "X26", "X50", "X3112"]

# Filter data
upper_values = {}
for target in targets:
    upper_values[target] = tabular_data[target+"_mean"].quantile(0.99)
    tabular_data = tabular_data[tabular_data[target+"_mean"] < upper_values[target]]
    tabular_data = tabular_data[tabular_data[target+"_mean"] > 0]

# Normalize the targets
original_means = tabular_data[[f"{target}_mean" for target in targets]].mean()
original_stds = tabular_data[[f"{target}_mean" for target in targets]].std()
tabular_data[[f"{target}_mean" for target in targets]] = (tabular_data[[f"{target}_mean" for target in targets]] - original_means) / original_stds

# Normalize the features
tabular_input_size = 0
for column in tabular_data.columns:
    if column in ["id"]+[target+"_mean" for target in targets]+[target+"_sd" for target in targets]:
        continue
    tabular_input_size += 1
    min_val = tabular_data[column].min()
    max_val = tabular_data[column].max()
    tabular_data[column] = (tabular_data[column] - min_val) / (max_val - min_val)


test_tabular_data = pd.read_csv(test_csv_path)
# Normalize the features
for column in test_tabular_data.columns:
    if column in ["id"]:
        continue
    min_val = test_tabular_data[column].min()
    max_val = test_tabular_data[column].max()
    test_tabular_data[column] = (test_tabular_data[column] - min_val) / (max_val - min_val)

train_images_dataset = ImageFolder(root=train_images_path, transform=transform)
test_images_dataset = ImageFolder(root=test_images_path, transform=transform)

train_image_csv_dataset = PGLSDataset(tabular_data=tabular_data, image_folder=train_images_dataset, transform_csv=None)
train, val = random_split(train_image_csv_dataset, [int(0.8*len(train_image_csv_dataset)), len(train_image_csv_dataset) - int(0.8*len(train_image_csv_dataset))])
test_image_csv_dataset = PGLSDataset(tabular_data=test_tabular_data, image_folder=test_images_dataset, transform_csv=None)


train_data_loader = DataLoader(train, batch_size=batch_size, shuffle=True)
val_data_loader = DataLoader(val, batch_size=batch_size, shuffle=True)
test_data_loader = DataLoader(test_image_csv_dataset, batch_size=batch_size, shuffle=False)

In [6]:
original_stds = torch.from_numpy(original_stds.values).float()
original_means = torch.from_numpy(original_means.values).float()
def denormalize_targets(targets):
    return targets * original_stds + original_means

In [7]:
# effnet = efficientnet_b0(weights=EfficientNet_B0_Weights)
effnet = timm.create_model(
    'efficientnet_b3.ra2_in1k',
    pretrained=True,
    num_classes=0,
)

# beit = timm.create_model('beit_base_patch16_224', pretrained=True, num_classes=0)
tabular_model = SimpleTabularModel(input_data_len=tabular_input_size)
model = PGLSModel(effnet, tabular_model)
metric = R2Score()
criterion = torch.nn.MSELoss()
torch.cuda.empty_cache()

In [8]:
model.to(device)

optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())

Epoch 0


RuntimeError: cuDNN error: CUDNN_STATUS_NOT_INITIALIZED

In [12]:
accumulated_loss = 0
iterations = 0
accumulated_r2 = 0
model.to(device)
model.eval()
with torch.no_grad():
    for data in val_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        loss = criterion(outputs, targets)
        accumulated_loss += loss.item()
        iterations += 1
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        accumulated_r2 += metric.compute().item()
print("Average loss on validation set: ", accumulated_loss/iterations)
print("Average R2 on validation set: ", accumulated_r2/iterations)

Average loss on validation set:  2.2444624650272433
Average R2 on validation set:  -1.1852780087781272


In [9]:
predictions = []
model.eval()
model.to(device)
with torch.no_grad():
    for data in test_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        outputs = model(image, features)
        outputs = outputs.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        predictions.append(outputs_denorm)
predictions = [item for sublist in predictions for item in sublist]

In [12]:
def save_predictions(predictions, test_csv_dataframe, filename):
    with open(filename, "w") as f:
        f.write("id,X4,X11,X18,X26,X50,X3112\n")
        for pred, id in zip(predictions, test_csv_dataframe["id"]):
            pred = [p.item() for p in pred]
            f.write(f"{id},{','.join([str(p) for p in pred])}\n")

In [38]:
save_predictions(predictions, test_tabular_data, "predictions.csv")

NameError: name 'save_predictions' is not defined

In [43]:
effnet = efficientnet_b0(weights=EfficientNet_B0_Weights).to(device)
xception = timm.create_model('ens_adv_inception_resnet_v2', pretrained=True, num_classes=1000).to(device)
densenet = torch.hub.load('pytorch/vision:v0.10.0', 'densenet121', pretrained=True).to(device)
# tabular_model = SimpleTabularModel(input_data_len=tabular_input_size).to(device)
tabular_model = deepfm.DenseArch(tabular_input_size, 100, 100).to(device)

/home/szaryvip/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=EfficientNet_B0_Weights.IMAGENET1K_V1`. You can also use `weights=EfficientNet_B0_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
/home/szaryvip/.local/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name ens_adv_inception_resnet_v2 to current inception_resnet_v2.tf_ens_adv_in1k.
  model = create_fn(
Using cache found in /home/szaryvip/.cache/torch/hub/pytorch_vision_v0.10.0
/home/szaryvip/.local/lib/python3.10/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/home/szaryvip/.local/lib/python3.10/site-pac

In [44]:

metric = R2Score()
criterion = torch.nn.MSELoss()
ensemble_model = EnsemblePGLSModel([effnet, xception, densenet], tabular_model)
torch.cuda.empty_cache()

In [47]:
ensemble_model.to(device)

optimizer = torch.optim.Adam(ensemble_model.parameters(), lr=0.001)

ensemble_model.train()
for epoch in range(2):
    print(f"Epoch {epoch}")
    for data in train_data_loader:
        image, features, targets = data
        image = image.to(device)
        features = features.to(device)
        targets = targets.to(device)
        optimizer.zero_grad()
        outputs = ensemble_model(image, features)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()
        outputs = outputs.to("cpu")
        targets = targets.to("cpu")
        outputs_denorm = denormalize_targets(outputs)
        targets_denorm = denormalize_targets(targets)
        metric.update(outputs_denorm, targets_denorm)
        print(loss.item())
        print(metric.compute())

Epoch 0
1576093.375
tensor(-0.7377)
814426.3125
tensor(-0.7467)
224598.25
tensor(-0.7492)
219645.1875
tensor(-0.7399)
906061.8125
tensor(-0.7478)
1727129.125
tensor(-0.7582)
2740938.0
tensor(-0.7741)
707776.0
tensor(-0.7634)
570188.0625
tensor(-0.7529)
1051340.5
tensor(-0.7540)
4920303.0
tensor(-0.7655)
175716.4375
tensor(-0.7743)
536888.6875
tensor(-0.7874)
1362061.375
tensor(-0.8085)
563966.5
tensor(-0.8157)
1004468.1875
tensor(-0.8150)
2490296.5
tensor(-0.8166)
754982.6875
tensor(-0.8152)
2201367.0
tensor(-0.8136)
998583.8125
tensor(-0.8164)
263361.03125
tensor(-0.8233)
2644947.75
tensor(-0.8234)
745026.4375
tensor(-0.8286)
1524908.375
tensor(-0.8477)
1759351.5
tensor(-0.8605)
6308760.0
tensor(-0.8643)
963321.9375
tensor(-0.8605)
145856.90625
tensor(-0.8639)
966126.8125
tensor(-0.8628)
13082564.0
tensor(-0.8755)
1459983.0
tensor(-0.8878)
1750251.625
tensor(-0.8867)
446590.0
tensor(-0.8886)
970637.6875
tensor(-0.8841)
2467010.5
tensor(-0.8932)
1943586.875
tensor(-0.8991)
1010202.75
t

KeyboardInterrupt: 